In [188]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Plan, Task, Language, SourceRange, Program, TASKS
from code_widget.example import CodeWidget
from dataclasses import replace
import json
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [189]:
task_id = 'unique_beer_drinkers'
task = TASKS[task_id]

In [217]:
program = Program(
    task=task_id,
    language='sql',
    author='will',
    plan={},
    source='''
SELECT DISTINCT L1.name
FROM likes L1
WHERE NOT EXISTS(
    SELECT *
    FROM likes L2
    WHERE L1.name != L2.name
    AND NOT EXISTS(
        SELECT *
        FROM likes L3
        WHERE L3.name = L2.name
        AND NOT EXISTS(
            SELECT *
            FROM likes L4
            WHERE L4.name = L1.name
            AND L4.beer = L3.beer))
    AND NOT EXISTS(
        SELECT *
        FROM likes L5
        WHERE L5.name = L1.name
        AND NOT EXISTS(
            SELECT *
            FROM likes L6
            WHERE L6.name = L2.name
            AND L6.beer= L5.beer)))
''')
program.execute(task)
program.save()

In [239]:
program = Program(
    task=task_id,
    language='sql',
    author='will',
    plan={},
    source='''
SELECT DISTINCT L1.name
FROM likes L1
WHERE NOT EXISTS(
    SELECT *
    FROM likes L2
    WHERE L1.name != L2.name
    AND NOT EXISTS(
        SELECT *
        FROM 
          (SELECT beer FROM likes WHERE name = L1.name
          UNION
          SELECT beer FROM likes WHERE name = L2.name) L5
        GROUP BY 
        HAVING COUNT(*) > 0
    ))
''')
program.execute(task)
program.save()

OperationalError: a GROUP BY clause is required before HAVING

In [44]:
program = Program(
    task=task_id,
    language='datalog',
    author='will',
    plan={},
    source='''
unique_beer_drinkers(name) :-
  likes(_, name),
  0 = count : {
    likes(_, other),
    other != name,
    shared_count = count : { likes(beer, other), likes(beer, name) },
    count : likes(_, other) = shared_count,
    count : likes(_, name) = shared_count
  }.
''')
program.execute(task)
program.save()

In [208]:
program = Program(
    task=task_id,
    language='python',
    author='will',
    plan={},
    source='''
import pandas as pd
def unique_beer_drinkers(likes):
  likes = pd.DataFrame(likes)
  likes_per_person = likes.groupby('name').beer.unique().map(set).reset_index()

  def check_not_exists(row):
      other_people = likes_per_person[likes_per_person.name != row['name']]
      return not (other_people.beer == row['beer']).any()

  return likes_per_person[likes_per_person.apply(check_not_exists, axis=1)].name.tolist()
''')
program.execute(task)
program.save()

In [213]:
program = Program(
    task=task_id,
    language='python',
    author='will',
    plan={},
    source='''
from collections import defaultdict    
def unique_beer_drinkers(likes):
  likes_per_person = defaultdict(set)
  for row in likes:
    likes_per_person[row['name']].add(row['beer'])
    
  unique_drinkers = []
  for name, beers in likes_per_person.items():
    unique = True
    for other_name, other_beers in likes_per_person.items():
        if other_name != name and beers == other_beers:
            unique = False
    if unique:
      unique_drinkers.append(name)
      
  return unique_drinkers
''')
program.execute(task)